In [110]:
import tkinter
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from tkinter import filedialog
import copy
import traceback
import sys

In [111]:
sys.version

'3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]'

In [18]:
def get_steam_library():
    tkinter.Tk().withdraw() # prevents an empty tkinter window from appearing
    folder_path = filedialog.askdirectory()
    return folder_path

In [9]:
path = 'C:\\Users\\Adria\\Documents\\Python_Projekt\\Wh40k - Gladius - Game Files\\Units\\ChaosSpaceMarines'
if os.getcwd() != path:
    os.chdir(path)

os.getcwd()

'C:\\Users\\Adria\\Documents\\Python_Projekt\\Wh40k - Gladius - Game Files\\Units\\ChaosSpaceMarines'

In [77]:
# path = get_steam_library()
#unit_filepath = path + '\\steamapps\\common\\Warhammer 40000 Gladius - Relics of War\\Data\\World\\Units\\ChaosSpaceMarines'
#icon_filepath = path + '\\steamapps\\common\\Warhammer 40000 Gladius - Relics of War\\Data\\Video\\Textures\\Icons\Attributes'

# unit_filepath = 'S:\\SteamLibrary\\steamapps\\common\\Warhammer 40000 Gladius - Relics of War\\Data\\World\\Units\\ChaosSpaceMarines'
# icon_filepath = 'S:\\SteamLibrary\\steamapps\\common\\Warhammer 40000 Gladius - Relics of War\\Data\\Video\\Textures\\Icons\Attributes'
# os.chdir(unit_filepath)
# os.getcwd()

### How to store data from soup
* `Object` + attributes according to the keywords (no dict)
* `String` or `List` of Strings
* `Dictionary` with keys according to the keywords

Storing informaton is only temporary.<br>
Goal is to store the data in a `DataFrame`.<br>
Memory utilization/usage from best to worst:<br>
`List` > `Object` > `Dictionary`

In [78]:
# Unit stats and cost keywords as used in the xml files
unit_stats = ['name','armor', 'hitpointsMax',
              'moraleMax','movementMax',
              'size', 'itemSlots',
              'cargoSlots']
unit_costs = ['productionCost','foodCost',
              'oreCost', 'energyCost',
              'influcenceCost','foodUpkeep',
              'oreUpkeep', 'energyUpkeep',
              'influenceUpkeep']
unit_attributes = unit_stats + unit_costs

In [79]:
def edit_list(data:list):
    """
    Edit list containing strings.
    Inserts spaces before capitalized letters.
    Then capitalizes the first letter of each word.
    Removes 'Max' from each String.    
    """
    data_copy = copy.deepcopy(data)
    for index, elem in enumerate(data_copy):
        data_copy[index] = re.sub(r"(\w)([A-Z])", r"\1 \2", elem).title().replace(' Max','')
    return data_copy

In [80]:
edit_unit_attrs = edit_list(unit_attributes)
print(edit_unit_attrs)

['Name', 'Armor', 'Hitpoints', 'Morale', 'Movement', 'Item Slots', 'Cargo Slots', 'Size', 'Production Cost', 'Food Cost', 'Ore Cost', 'Energy Cost', 'Influcence Cost', 'Food Upkeep', 'Ore Upkeep', 'Energy Upkeep', 'Influence Upkeep']


In [115]:
def create_soups():
    """Function creates a list containing the soup of each xml file"""
    os.chdir('C:\\Users\\Adria\\Documents\\Python_Projekt\\Wh40k - Gladius - Game Files\\Units\\ChaosSpaceMarines')
    pot = []
    files = os.listdir()
    for file in files:
        if file == 'Headquarters.xml':
            continue
        with open(file, 'r') as f:
            data = f.read()
            soup = BeautifulSoup(data, 'xml')
            pot.append([soup, file])
    os.chdir('C:\\Users\\Adria\\Documents\\Python_Projekt\\Wh40k - Gladius - Game Files\\')
    return pot
pot = create_soups() 

In [82]:
def get_unit_stats(soup, filename):
    """
    Generates list containing unit stats as Strings.
    Iterates over unit_stats keyword list and looks for 
    identical tags in XML file.
    """
    stats = []
    # required soups
    name = filename.split('.')[0]
    group = soup.find_all('group')
    if name == "NoctilithCrown":
        effects = soup.find_all('effects')[1]
    else:
        effects = soup.find_all('effects')[0]
    # For structure units values at index 1
    
    for index, s in enumerate(unit_attributes):
        try:
            if s=='name':
                # edits unit name for increased readability
                stats.append(re.sub(r"(\w)([A-Z])", r"\1 \2", name))
            elif s=='size': 
                stats.append(int(group[0].attrs['size']))
            else:
                attr = effects.find(s).attrs['base'].split('.')[0]
                stats.append(int(attr))
    
        except IndexError: # caused by group
            stats.append(1)
            
        except KeyError: # If attribute key does not exist, fill with None value
            # print(traceback.format_exc())
            stats.append(0) 
        
        except AttributeError: # caused by effect if the tag does not exist
            # print(traceback.format_exc())
            stats.append(0)
        
    return stats         

In [83]:
def test_unit_stats(idx):
    unit = get_unit_stats(pot[idx][0], pot[idx][1])
    print(unit)
    print("Length: ", len(unit), '\n')
test_unit_stats(-5)

['Noctilith Crown', 8, 24, 8, 0, 0, 0, 1, 36, 0, 20, 20, 0, 0, 0, 2, 0]
Length:  17 



### Create `DataFrame` from unit list with keywords for columns

In [84]:
# Function to iterate of each XML file and process its soup
# Return an object used for the generation of the dataframe
def create_unit_table():
    """
    Iteratively extracts unit data from the XML file and stores it in a list.
    Every unit data list is stored in unit list (nested list).
    Returns unit list.
    """
    unit_list = []
    files = os.listdir()
    for file in files:
        if file == "Headquarters.xml":
            continue
        try:
            with open(file, 'r') as f: 
                data = f.read()
                soup = BeautifulSoup(data, 'xml')
                unit_list.append(get_unit_stats(soup, file))
        except PermissionError:
            continue
    return unit_list

In [85]:
unit_data = create_unit_table()
unit_index = list(range(len(unit_data)))
pd.DataFrame(index=unit_index, data=unit_data, columns=edit_list(unit_attributes))

,Name,Armor,Hitpoints,Morale,Movement,Item Slots,Cargo Slots,Size,Production Cost,Food Cost,Ore Cost,Energy Cost,Influcence Cost,Food Upkeep,Ore Upkeep,Energy Upkeep,Influence Upkeep
0,Chaos Cultist,2,2,4,3,0,0,10,24,10,10,0,0,1,0,0,0
1,Chaos Land Raider,10,64,12,4,0,3,1,60,0,160,0,0,0,8,0,0
2,Chaos Lord,8,9,12,3,6,0,1,42,30,30,0,0,3,0,0,6
3,Chaos Rhino,7,24,10,4,0,3,1,30,0,30,0,0,0,1,0,0
4,Chaos Space Marine,8,3,8,3,0,0,5,36,20,20,0,0,2,0,0,0
5,Chaos Spawn,0,12,12,5,0,0,3,36,20,0,20,0,2,0,0,0
6,Daemon Prince,8,20,12,3,6,0,1,42,30,0,30,0,0,0,3,4
7,Dark Disciple,4,2,6,3,0,0,10,30,15,15,0,0,1,0,0,0
8,Defiler,8,48,10,3,0,0,1,60,0,160,0,0,0,8,0,0
9,Forgefiend,8,36,10,3,0,0,1,54,0,120,0,0,0,6,0,0


In [19]:
def get_all_faction_units():
    # path is global variable for steamlibrary directory
    
    # os.chdir(path + )
    os.chdir('C:\\Users\\Adria\\Documents\\Python_Projekt\\Wh40k - Gladius - Game Files\\Units')
    path = os.getcwd()
    factions = os.listdir()
    #iterate through every faction folder
    df_list = []
    for idx, folder in enumerate(factions):
        # enter faction folder
        os.chdir(path + "\\" + folder)
        # get unit data (iterates through every file)
        unit_data = create_unit_table()
        unit_index = list(range(len(unit_data)))
        df_list.append(pd.DataFrame(index=unit_index, data=unit_data, columns=edit_list(unit_attributes)))
     
    complete_df = pd.concat(df_list, ignore_index=True)
    return complete_df
    
# get_all_faction_units()

In [87]:
def save_df(df):
    os.chdir('C:\\Users\\Adria\\Documents\\Python_Projekt')
    try:
        os.remove("unit_stats.xlxs")
        os.remove("unit_stats.csv")
    except:
        pass
    df.to_csv("unit_stats.csv", encoding='utf-8', index=False)
    df.to_excel("unit_stats.xlsx")

save_df(get_all_faction_units())

### Further data extraction

In [37]:
def read_trait(file):
    """
    Enter trait file and access the name.
    File - XML file in the current directory

    returns - String, String: name and category of the trait stored in the file
    """
    try:
        with open(file, 'r') as f:
            soup = BeautifulSoup(f.read(), 'xml')
            # get name from file name
            category = soup.find('trait')[0].attrs['category']
            return re.sub(r"(\w)([A-Z])", r"\1 \2", file), category
    except:
        # throw new exception for catch/exception in loop
        print(file + " not found!")
        # what to do if an Error occcurs?

    
def get_trait_list():
    try:
        # path from get_SteamLibrary 
        # os.chdir(path + \\steamapps\\common\\Warhammer 40000 Gladius - Relics of War\\Data\\World\\)
        os.chdir('C:\\Users\\Adria\\Documents\\Python_Projekt\\Wh40k - Gladius - Game Files\\Traits')
    except:
        print(traceback.format_exc())
    
    # change directory to 'trait'
    traits = []
    # Directory is filled with "neutral" files and faction specific traits
    # List for with name with faction and upgrade requirement???
    neutral_traits = []
    print("We start here: ", os.getcwd())
    for file in os.listdir():
        if os.path.isdir(file):
            faction_traits = [] # Trait list for current faction
            faction_name = file 
            current_path = os.getcwd() # store current path to change back to
            os.chdir(current_path + '\\' + file) # change directory to faction folder
            for file in os.listdir():
                if not os.path.isdir(file):
                    try: 
                        with open(file, 'r') as f:
                            soup = BeautifulSoup(f.read(), 'xml')
                            # get name from file name
                            category = soup.find('trait')[0].attrs['category']
                            trait = re.sub(r"(\w)([A-Z])", r"\1 \2", file).split('.')[0]
                            faction_traits.append([trait, category]) # for every trait store name + faction
                    except:
                        continue
            traits.append(faction_traits)
            os.chdir(current_path)
        else:
            try:
                trait = re.sub(r"(\w)([A-Z])", r"\1 \2", file).split('.')[0]
                neutral_traits.append([trait, 'Neutral']) # for every trait store name + faction
            except:
                continue
    traits.append(neutral_traits)
    return traits

trait_list = get_trait_list()

We start here:  C:\Users\Adria\Documents\Python_Projekt\Wh40k - Gladius - Game Files\Traits


In [146]:
traits = []
counts = []
os.chdir('C:\\Users\\Adria\\Documents\\Python_Projekt\\Wh40k - Gladius - Game Files\\Traits')
for file in os.listdir():
    if os.path.isdir(file):
        counter = 0
        faction_traits = [] # Trait list for current faction
        faction_name = re.sub(r"(\w)([A-Z])", r"\1 \2", file) 
        # print(file, " - ", faction_name)
        
        current_path = os.getcwd() # store current path to change back to
        os.chdir(current_path + '\\' + file) # change directory to faction folder
        
        for file in os.listdir():
            if not os.path.isdir(file):
                counter += 1
                file_name = re.sub(r"(\w)([A-Z])", r"\1 \2", file).split('.')[0]
                traits.append([file_name, faction_name])
                # collect file names - explicit human readable information required
                # further research in other game files necessary
                
        counts.append(counter)
        os.chdir(current_path)

        """
            if not os.path.isdir(file):
                try: 
                    with open(file, 'r') as f:
                        soup = BeautifulSoup(f.read(), 'xml')
                        # here lies the problem
                        category = soup.find('trait')[0].attrs['category']
                        
                        trait = re.sub(r"(\w)([A-Z])", r"\1 \2", file).split('.')[0]
                        faction_traits.append([trait, category]) # for every trait store name + faction
                        counter -=1
                except:
                    continue
                    
        traits.append(faction_traits)
        """
print(counts)

[58, 6, 6, 54, 51, 18, 45, 34, 10, 48, 48]


In [106]:
trait_idx = list(range(len(traits)))
trait_col = ['Trait_name', 'Origin']
df = pd.DataFrame(index=trait_idx, data=traits, columns=trait_col)
df.loc[df['Origin'] == 'Adeptus Mechanicus']
col = list(set(df.Origin.values))
col.sort()
for c in col:
    print(df.loc[df['Origin'] == c])

                       Trait_name              Origin
0           Adjacency Integration  Adeptus Mechanicus
1             Aggression Override  Adeptus Mechanicus
2            Aggressor Imperative  Adeptus Mechanicus
3            Assault Weapon Bonus  Adeptus Mechanicus
4      Benediction Of Omniscience  Adeptus Mechanicus
5                         Bionics  Adeptus Mechanicus
6      Broad Spectrum Data Tether  Adeptus Mechanicus
7              Bulwark Imperative  Adeptus Mechanicus
8      Canticles Of The Omnissiah  Adeptus Mechanicus
9   Chant Of The Remorseless Fist  Adeptus Mechanicus
10                     City Tier2  Adeptus Mechanicus
11                     City Tier3  Adeptus Mechanicus
12                         Cognis  Adeptus Mechanicus
13                 Command Uplink  Adeptus Mechanicus
14           Conqueror Imperative  Adeptus Mechanicus
15                  Control Edict  Adeptus Mechanicus
16                Darting Hunters  Adeptus Mechanicus
17        Data Blessed Autos

In [116]:
os.getcwd()

'C:\\Users\\Adria\\Documents\\Python_Projekt\\Wh40k - Gladius - Game Files'

In [131]:
def get_unit_traits(soup):
    """
    Iterates over each trait tag in soup.
    Return list with all trait names.
    If trait requires an upgrade, keyword is added to name.
    """
    traits = []
    soup = soup.find_all('trait')
    for trait in soup:
        try: # catching a KeyError. Not every trait has the attribute requiredUpgrade
            name = trait.attrs['name'].split('/')[-1]
            if (trait.attrs['requiredUpgrade']): 
                upgrade_name = trait.attrs['requiredUpgrade'].split('/')[-1]
                traits.append([name, upgrade_name])
            else:
                traits.append([name, None])
        except:
            continue
    return traits

In [137]:
# test - get_unit_traits()
a = get_unit_traits(pot[1][0])
b = pot[1][1]
print(a, b)

[['ExtraVehicleArmour', 'ExtraVehicleArmour'], ['SoulBlaze', 'WarpflameGargoyles']] ChaosLandRaider.xml


In [139]:
def get_action_names(soup):
    action_names = [] # names
    actions = {} # name - details
    action_soup = [value for value in soup.find('actions').contents if value != '\n']
    for a in action_soup:
        value = a.name
        if a.name == 'throwGrenade':
            value = a.attrs['weaponSlotName']
        if value == 'genericUnitAbility':
            value = a.attrs['name'].split('/')[1]
        action_names.append(value)
    return action_names

In [142]:
get_action_names(pot[0][0])

['attack',
 'die',
 'idle',
 'move',
 'foundCity',
 'clearTileUnitAbility',
 'cultistSacrifice',
 'constructNoctilithCrown']

In [ ]:
def get_weapon_names(soup):
    attributes = []
    for entry in soup.find_all(name='weapon'):
        try:
            name = None
            name = entry.attrs['name']
            if entry.attrs['requiredUpgrade']: attributes.append("Upgrade-" + entry.attrs['name'])
        except:
            if name: attributes.append(name)
            else: continue
    return attributes

def get_weapon_id(weapon_df, weapon_list):
    for weapon in weapon_list:
        weapon_df['weapon_list']

In [ ]:
# test - get_weapon_names()
get_weapon_names(pot[1])

In [ ]:
# Edit_list replaces edit_dict
# Keys are getting editet in get_unit_stats
def edit_dict(data):
    for key in data.keys():
        key = re.sub(r"(\w)([A-Z])", r"\1 \2", key).title
    return data

In [ ]:
# Function to call all data extraction function on a soup
def extract_data(soup):
    stats = get_unit_stats(soup) # dict
    weapons = get_weapon_names(soup) # list
    actions = get_action_names(soup) # list
    stats['weapons'] = weapons
    stats['actions'] = actions
    return stats

In [ ]:
cultist = get_unit_stats(pot[2])
c = {'name':"A",'test':0}
pd.DataFrame(data=[[0,0],[0,0]],columns=["a","b"])
pd.DataFrame(cultist, [0]) #unit_attributes)
# pd.DataFrame(edit_dict(cultist),[0])